# IMPORTS

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

from scipy.fft import fft, fftfreq

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBRegressor


RANDOM_STATE = 420

from google.colab import drive
drive.mount('/content/drive')

# Load Data

In [2]:
path = "G:\My Drive\AW_Academy\mini_project_week6\github repo\df/"
path2= '/data/'
df=pd.read_csv(f'{path}df.csv')

# Features

In [5]:
# Curr month sales (y)
month_sales = df[["date_block_num", "item_cnt_day", "revenue"]].copy()

temp = month_sales.groupby(by="date_block_num", as_index=False)
temp = temp.sum()
temp.columns=["date_block_num", "item_cnt_month", "revenue_month"]

sales = df.merge(temp, on="date_block_num", how="left")

## Prev month lag

In [6]:
# Prev month sales and revenue
prev_month_sales = sales[["date_block_num", "item_cnt_day", "revenue"]].copy()
prev_month_sales["date_block_num"] += 1

temp = prev_month_sales.groupby(by="date_block_num", as_index=False)
temp = temp.sum()
temp.columns=["date_block_num", "item_cnt_pri_1_month", "revenue_pri_1_month"]

sales = sales.merge(temp, on="date_block_num", how="left")

In [7]:
# 2 months ago sales and revenue
prev_month_sales = sales[["date_block_num", "item_cnt_day", "revenue"]].copy()
prev_month_sales["date_block_num"] += 2

temp = prev_month_sales.groupby(by="date_block_num", as_index=False)
temp = temp.sum()
temp.columns=["date_block_num", "item_cnt_pri_2_month", "revenue_pri_2_month"]

sales = sales.merge(temp, on="date_block_num", how="left")

In [8]:
# 3 months ago sales and revenue
prev_month_sales = sales[["date_block_num", "item_cnt_day", "revenue"]].copy()
prev_month_sales["date_block_num"] += 3

temp = prev_month_sales.groupby(by="date_block_num", as_index=False)
temp = temp.sum()
temp.columns=["date_block_num", "item_cnt_pri_2_month", "revenue_pri_2_month"]

sales = sales.merge(temp, on="date_block_num", how="left")

In [9]:
# 6 months ago sales and revenue
prev_month_sales = sales[["date_block_num", "item_cnt_day", "revenue"]].copy()
prev_month_sales["date_block_num"] += 6

temp = prev_month_sales.groupby(by="date_block_num", as_index=False)
temp = temp.sum()
temp.columns=["date_block_num", "item_cnt_pri_2_month", "revenue_pri_2_month"]

sales = sales.merge(temp, on="date_block_num", how="left")

del temp
del prev_month_sales

## Time since first sale by month - NaN is present due to test month not having a value, which is filled with 34

In [10]:
sales["first_sale_date"] = sales.groupby("item_id").agg({"date_block_num":"min"})["date_block_num"]
sales[sales["first_sale_date"].isna()]
sales["first_sale_date"] = sales["first_sale_date"].fillna(34)
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dof,doy,dom,revenue,item_category_id,item_name_first_4,item_name_first_6,group_name_id,item_cnt_month,revenue_month,item_cnt_pri_1_month,revenue_pri_1_month,item_cnt_pri_2_month_x,revenue_pri_2_month_x,item_cnt_pri_2_month_y,revenue_pri_2_month_y,item_cnt_pri_2_month,revenue_pri_2_month,first_sale_date
0,2013-01-02 00:00:00,0,59,22154,999.0,1.0,2.0,2.0,2.0,999.0,37,3795,5819,10,126400.0,78120899.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
1,2013-01-23 00:00:00,0,24,22154,999.0,1.0,2.0,23.0,23.0,999.0,37,3795,5819,10,126400.0,78120899.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
2,2013-01-20 00:00:00,0,27,22154,999.0,1.0,6.0,20.0,20.0,999.0,37,3795,5819,10,126400.0,78120899.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0
3,2013-01-02 00:00:00,0,25,22154,999.0,1.0,2.0,2.0,2.0,999.0,37,3795,5819,10,126400.0,78120899.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0
4,2013-01-03 00:00:00,0,25,22154,999.0,1.0,3.0,3.0,3.0,999.0,37,3795,5819,10,126400.0,78120899.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073750,0,34,46,12470,0.0,0.0,0.0,0.0,0.0,0.0,58,2158,3281,12,0.0,0.00,57855.0,58633335.16,55205.0,54871467.06,61230.0,51091303.73,58457.0,52586761.7,34.0
3073751,0,34,41,12470,0.0,0.0,0.0,0.0,0.0,0.0,58,2158,3281,12,0.0,0.00,57855.0,58633335.16,55205.0,54871467.06,61230.0,51091303.73,58457.0,52586761.7,34.0
3073752,0,34,44,12470,0.0,0.0,0.0,0.0,0.0,0.0,58,2158,3281,12,0.0,0.00,57855.0,58633335.16,55205.0,54871467.06,61230.0,51091303.73,58457.0,52586761.7,34.0
3073753,0,34,39,12470,0.0,0.0,0.0,0.0,0.0,0.0,58,2158,3281,12,0.0,0.00,57855.0,58633335.16,55205.0,54871467.06,61230.0,51091303.73,58457.0,52586761.7,34.0


# Export data

In [11]:
compression_opts = dict(method='zip',
                        archive_name='df_feat.csv')  

sales.to_csv('df_feat.zip', index=False,
          compression=compression_opts)  